# Entrenamiento de una CNN en Pytorch

- El entrenamiento de una red neuronal convolucional (CNN) en PyTorch implica varios pasos importantes

### 1. Preparación de los datos: Dividir los datos en conjuntos de entrenamiento, validación y prueba utilizando las funciones de PyTorch, aplicar preprocesamiento y transformaciones necesarias.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Set random seed for reproducibility
torch.manual_seed(0)

# Define the transforms for data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the image tensors
])

# Load the CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create data loaders for batch processing
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


### 2. Definición del modelo: Crear una clase que herede de `nn.Module` y definir la arquitectura de la CNN utilizando los módulos proporcionados por PyTorch, como `nn.Conv2d` y `nn.Linear`.

In [2]:
# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(32 * 8 * 8, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
# Create an instance of the CNN model
model = CNN()

### 3. Función de pérdida y optimización: Seleccionar la función de pérdida adecuada de `torch.nn` según el tipo de problema y elegir un optimizador de `torch.optim` para ajustar los pesos de la red, como `torch.optim.Adam` o `torch.optim.SGD`.

In [3]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 4. Ciclo de entrenamiento: Iterar sobre los datos de entrenamiento en lotes, calcular la salida de la CNN, calcular la pérdida utilizando la función de pérdida elegida, propagar hacia atrás para calcular los gradientes y actualizar los pesos mediante el optimizador.

In [4]:
# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # 5. Validación y ajuste: Evaluar el rendimiento del modelo en el conjunto de validación, realizar ajustes en los hiperparámetros y prevenir el sobreajuste utilizando técnicas como la regularización o el ajuste de la tasa de aprendizaje.
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Test Accuracy: {accuracy:.2f}%')

Epoch 1/10, Test Accuracy: 55.73%
Epoch 2/10, Test Accuracy: 61.80%
Epoch 3/10, Test Accuracy: 64.82%
Epoch 4/10, Test Accuracy: 66.81%
Epoch 5/10, Test Accuracy: 67.74%
Epoch 6/10, Test Accuracy: 68.76%
Epoch 7/10, Test Accuracy: 68.65%
Epoch 8/10, Test Accuracy: 69.05%
Epoch 9/10, Test Accuracy: 69.90%
Epoch 10/10, Test Accuracy: 70.03%


In [5]:

# Save the trained model
torch.save(model.state_dict(), 'cifar10_cnn_model.pth')


# Ejercicio: Probar distintas funciones de pérdida y optimizadores para ver el impacto de los mismos